In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Setup Distiller

In [ ]:
!git clone https://github.com/mueedurrehman/distiller.git
%cd distiller
!pip3 install -e .

fatal: destination path 'distiller' already exists and is not an empty directory.
/content/distiller
Obtaining file:///content/distiller
  Found existing installation: distiller 0.4.0rc0
    Can't uninstall 'distiller'. No files were found to uninstall.
  Running setup.py develop for distiller


In [ ]:
!pip install xdelta3

# Compute Statistic

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torchsummary import summary
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
# import distiller
# import distiller.models
# import xdelta3

import time
import math
import copy
import zipfile
from enum import Enum
import zipfile
import sys
import glob


from collections import OrderedDict
import os
import pandas as pd

In [ ]:
def extractIndicesVals(vals):
  matrix = copy.deepcopy(vals)
  matrix = matrix.numpy()
  values = np.ravel(matrix)
  indices = (values != 0).astype(np.int8)
  values = values[values != 0]
  indicesPacked = np.packbits(indices)
  return indicesPacked, values

def computeStorageTensor(name, tensor,zipfile):
  indicesPacked, values = extractIndicesVals(tensor)
  totalSize = 0
  np.save(name + "I.npy", indicesPacked)
  np.save(name + "V.npy", values)
  zipfile.write(name + "I.npy")
  zipfile.write(name + "V.npy")
  return totalSize

def deltaEncodeIndicesValues(indices, values):
  indicesDelta = []
  valsDelta = []
  if indices[0] > 8:
    numZeros = indices[0] // 8
    for i in range(numZeros):
      valsDelta.append(0)
      #append a 7, which corresponds to a delta of 8
      indicesDelta.append(7)
    indicesDelta.append((indices[0] % 8) - 1)
    #now add the value at this index
    valsDelta.append(values[0])
  #first value has been handled, compute differences and handle those
  indices = np.diff(indices)
  for i in range(len(indices)):
    index = indices[i]
    if index > 8:
      numZeros = index // 8
      for j in range(numZeros):
        valsDelta.append(0)
        #append a 7, which corresponds to a delta of 8
        indicesDelta.append(7)
      indicesDelta.append((index % 8) - 1)
      #now add the value at this index
      valsDelta.append(values[i + 1])
    else:
      indicesDelta.append(index - 1)
      valsDelta.append(values[i + 1])
  return np.array(indicesDelta, dtype = np.int8), np.array(valsDelta, dtype = np.float32)

def convertDeltaIndicesToBits(deltaIndices):
  position = np.ones(deltaIndices.shape, dtype = np.int8) * 4
  result = None
  for i in range(3):
    bits = np.bitwise_and(deltaIndices, position)
    bits = np.right_shift(bits, 2 - i)
    if result is None:
      result = bits.reshape(1,-1)
    else:
      result = np.concatenate((result,bits.reshape(1,-1)), axis = 0)
    position = np.right_shift(position, 1)
  # print("the length of indices in encoding is", np.ravel(result, order = 'F').shape)
  return np.packbits(np.ravel(result, order = 'F'))

def extractDeltaIndicesVals(vals):
  matrix = copy.deepcopy(vals)
  matrix = matrix.numpy()
  values = np.ravel(matrix)
  indices = np.argwhere(values != 0).flatten()
  values = values[values != 0]
  return deltaEncodeIndicesValues(indices, values)

def computeStorageTensorDelta(name, tensor, zipfile):
  indices, values = extractDeltaIndicesVals(tensor)
  indices = convertDeltaIndicesToBits(indices)
  totalSize = 0
  np.save(name + "I.npy", indices)
  np.save(name + "V.npy", values)
  zipfile.write(name + "I.npy")
  zipfile.write(name + "V.npy")

def processDictionary(dictionary, computeStorageFunction):
  totalSize = 0
  newDictionary = OrderedDict()
  zipTypes = [zipfile.ZIP_LZMA, zipfile.ZIP_DEFLATED]
  sizes = []
  for zipType in zipTypes:
    if not os.path.exists('archive.zip'):
      os.mknod('archive.zip')
    deflated = zipfile.ZipFile("archive.zip", mode = 'w', compression = zipType)
    for key,value in dictionary.items():
      newDictionary[key] = value.size()
      computeStorageFunction(key, value.cpu(), deflated)
    torch.save(newDictionary, "dictionaryTorch")
    deflated.write("dictionaryTorch")
    sizes.append(os.path.getsize("archive.zip"))
    os.remove("archive.zip")
  return sizes

def compressDictionary(dictionary, method):
  if method == "bitmap":
    return processDictionary(dictionary, computeStorageTensor)
  elif method == "deltaEncoding":
    return processDictionary(dictionary, computeStorageTensorDelta)

In [ ]:
#helper functions

transformTest = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

test = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform = transformTest)
testLoader = torch.utils.data.DataLoader(test, batch_size=256,
                                         shuffle=False, num_workers=2)
def to_sparse(x):
    """ converts dense tensor x to sparse format """
    x_typename = torch.typename(x).split('.')[-1]
    sparse_tensortype = getattr(torch.sparse, x_typename)

    indices = torch.nonzero(x)
    if len(indices.shape) == 0:  # if all elements are zeros
        return sparse_tensortype(*x.shape)
    indices = indices.t()
    values = x[tuple(indices[i] for i in range(indices.shape[0]))]
    return sparse_tensortype(indices, values, x.size())

def computeStatePyTorch(state_dict, compressed = True):
  originalSize = 0
  compressedSize = 0
  torch.save(state_dict, "./curStateDict")
  totalSize = os.path.getsize("./curStateDict")
  if compressed:
    for value in state_dict.values():
      torch.save(value, "denseTensor")
      sparse = to_sparse(value)
      torch.save(sparse, "sparse")
      originalSize += os.path.getsize("./denseTensor")
      compressedSize += os.path.getsize("./sparse")
    os.remove("./sparse")
    return totalSize - originalSize + compressedSize

def computeSize(stateDictionary):
  torch.save(stateDictionary, "stateDict")
  size = os.path.getsize("stateDict")
  os.remove("stateDict")
  return size

def computeTotalParameters(model):
  return sum(p.numel() for p in model.parameters())

def computeNonZeroParameters(model):
  return sum(p.nonzero().size(0) for p in model.parameters())

baseModelCheckpoint = torch.load("/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Baseline Models/baseResnet56")
baseModel = distiller.models.create_model(False, "cifar10", "resnet56_cifar", True, None)
baseModel.load_state_dict(baseModelCheckpoint["state_dict"])
baseParameters = computeTotalParameters(baseModel)
baseSize = computeSize(baseModel.state_dict())
# _, baseTime = test(baseModel)

def computeIndexSavingRate(baseParameters, compressedParameters):
  return (baseParameters - compressedParameters)/compressedParameters

#accuracy will not be accurate for the quantized models
def test(net):
    startTime = time.time()
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testLoader):
            inputs, targets = inputs.to('cuda'), targets.to('cuda')
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    acc = 100.*correct/total
    return acc, time.time() - startTime

def computeStatistics(compressedModel):
  modelParameters = computeNonZeroParameters(compressedModel)
  indexSavingRate = computeIndexSavingRate(baseParameters, modelParameters)
  parameterCompressionRate = baseParameters / modelParameters
  accuracy,time = test(compressedModel)
  accuracy = round(accuracy, 2)
  bitmapSizes = compressDictionary(compressedModel.state_dict(), "bitmap")
  deltaEncodingSizes = compressDictionary(compressedModel.state_dict(), "deltaEncoding")
  size_bitmap_LZMA = bitmapSizes[0]
  size_bitmap_DEFLATED = bitmapSizes[1]
  size_deltaEncoding_LZMA = deltaEncodingSizes[0]
  size_deltaEncoding_DEFLATED = deltaEncodingSizes[1]
  scr_bitmap_LZMA = baseSize / size_bitmap_LZMA
  scr_bitmap_DEFLATED = baseSize / size_bitmap_DEFLATED
  scr_deltaEncoding_LZMA = baseSize / size_deltaEncoding_LZMA
  scr_deltaEncoding_DEFLATED = baseSize / size_deltaEncoding_DEFLATED
 
  sparsity = round(distiller.utils.model_sparsity(compressedModel), 2)
  #bitmap sizes have 2 sizes, deltaEncoding has 2 sizes, first size is LZMA, second size is deflated, this means 4 size compression rates are computed
  #Add to return value 4 size compression rates as well as original sizes under each scheme
  # Returning 4 size compression rates, 4 sizes, accuracy, time, parameterCompressionRate, IndexSavingRate, modelParameters, sparsity
  # return {"Size Compression Rate" : sizeCompressionRate, "ParameterCompressionRate" : parameterCompressionRate, "IndexSavingRate": indexSavingRate}

  return {"Accuracy" : accuracy,
          "Sparsity" : sparsity,
          "Model Parameters" : modelParameters, 
          "Size_bitmap_LZMA" : size_bitmap_LZMA,
          "Size_bitmap_DEFLATED" : size_bitmap_DEFLATED,
          "Size_deltaEncoding_LZMA" : size_deltaEncoding_LZMA,
          "Size_deltaEncoding_DEFLATED" : size_deltaEncoding_DEFLATED,
          "SCR_bitmap_LZMA" : scr_bitmap_LZMA,
          "SCR_bitmap_DEFLATED" : scr_bitmap_DEFLATED,
          "SCR_deltaEncoding_LZMA" : scr_deltaEncoding_LZMA,
          "SCR_deltaEncoding_DEFLATED" : scr_deltaEncoding_DEFLATED,          
          "ParameterCompressionRate" : parameterCompressionRate, 
          "IndexSavingRate": indexSavingRate 
  }
          # "Train Parameter Compression Rate" : parameterCompressionRateTrain, 
          # "Train Index Saving Rate" : indexSavingRateTrain}

Files already downloaded and verified


In [ ]:
# Sun Dec 13 06:56:50 2020       
# +-----------------------------------------------------------------------------+
# | NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
# |-------------------------------+----------------------+----------------------+
# | GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
# | Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
# |                               |                      |               MIG M. |
# |===============================+======================+======================|
# |   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
# | N/A   76C    P0    34W /  70W |   1121MiB / 15079MiB |      0%      Default |
# |                               |                      |                 ERR! |
# +-------------------------------+----------------------+----------------------+
                                                                               
# +-----------------------------------------------------------------------------+
# | Processes:                                                                  |
# |  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
# |        ID   ID                                                   Usage      |
# |=============================================================================|
# |  No running processes found                                                 |
# +-----------------------------------------------------------------------------+
!nvidia-smi

Sun Dec 13 20:17:38 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0    35W /  70W |    851MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# bits_parameters, bit_acitvations, bit_weights clipAC qunatizer metadata
# in metadata
# post qa bitwidth 
# qa actvations and weights 

# Post

In [ ]:
# resultsDir = "/content/drive/MyDrive/logs/"
resultsDir = "/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Quantization Schedules/PostTrainingResults/2020.12.08-195132"
act_stats = "/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Quantization Schedules/acts_quantization_stats.yaml"

modelpost = distiller.models.create_model(False, "cifar10", "resnet56_cifar", True, None)
originalDictionary = copy.deepcopy(modelpost.state_dict())
quantizerpost = distiller.quantization.PostTrainLinearQuantizer(model = modelpost, bits_activations=8, bits_parameters= 8, bits_accum=32,
                                                            mode=distiller.quantization.LinearQuantMode.ASYMMETRIC_UNSIGNED, model_activation_stats= act_stats, per_channel_wts=True)
dummy_input = distiller.get_dummy_input(input_shape=modelpost.input_shape)
quantizerpost.prepare_model(dummy_input)

mapping = {}
mapping2 = {}

for file in os.listdir(resultsDir):
  # # print(file)
  # target = os.path.join(resultsDir, file) 
  # subfolder = os.listdir(target)[0]
  # target = os.path.join(target, subfolder, "quantized_checkpoint.pth.tar")
  target = os.path.join(resultsDir, "quantized_checkpoint.pth.tar")
  print(target)
  if os.path.exists(target):
    print("Hi")
    checkpoint = torch.load(target)
    modelpost.load_state_dict(checkpoint["state_dict"])
    sparsity = round(distiller.utils.model_sparsity((modelpost)), 2)
    checkpoint["extras"]
    acc = round(checkpoint["extras"]["quantized_top1"], 2)
    mapping[file] = str(acc)
    mapping2[file] = str(sparsity)
    str(sparsity)

df = pd.DataFrame.from_dict(mapping, orient = 'index')
df2 = pd.DataFrame.from_dict(mapping2, orient = 'index')

df

/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Quantization Schedules/PostTrainingResults/2020.12.08-195132/quantized_checkpoint.pth.tar
Hi
/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Quantization Schedules/PostTrainingResults/2020.12.08-195132/quantized_checkpoint.pth.tar
Hi
/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Quantization Schedules/PostTrainingResults/2020.12.08-195132/quantized_checkpoint.pth.tar
Hi
/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Quantization Schedules/PostTrainingResults/2020.12.08-195132/quantized_checkpoint.pth.tar
Hi


,0
2020.12.08-195132.log,40.96
quant_stats_after_prepare_model.yaml,40.96
layer_quant_params.yaml,40.96
quantized_checkpoint.pth.tar,40.96


In [ ]:
checkpoint

{'arch': 'resnet56_cifar',
 'compression_sched': {'masks_dict': {'module.bn1.bias': None,
   'module.bn1.weight': None,
   'module.conv1.weight': None,
   'module.fc.bias': None,
   'module.fc.weight': None,
   'module.layer1.0.bn1.bias': None,
   'module.layer1.0.bn1.weight': None,
   'module.layer1.0.bn2.bias': None,
   'module.layer1.0.bn2.weight': None,
   'module.layer1.0.conv1.weight': None,
   'module.layer1.0.conv2.weight': None,
   'module.layer1.1.bn1.bias': None,
   'module.layer1.1.bn1.weight': None,
   'module.layer1.1.bn2.bias': None,
   'module.layer1.1.bn2.weight': None,
   'module.layer1.1.conv1.weight': None,
   'module.layer1.1.conv2.weight': None,
   'module.layer1.2.bn1.bias': None,
   'module.layer1.2.bn1.weight': None,
   'module.layer1.2.bn2.bias': None,
   'module.layer1.2.bn2.weight': None,
   'module.layer1.2.conv1.weight': None,
   'module.layer1.2.conv2.weight': None,
   'module.layer1.3.bn1.bias': None,
   'module.layer1.3.bn1.weight': None,
   'module.lay

In [ ]:
bits_parameters = checkpoint['quantizer_metadata']["params"]["bits_parameters"]
clip_acs = checkpoint['quantizer_metadata']["params"]["clip_acts"]
mode = checkpoint['quantizer_metadata']["params"]["mode"]["activations"]
accur = round(checkpoint["extras"]["quantized_top1"], 2)
print(bits_parameters, clip_acs, mode, accur)

4 NONE ASYMMETRIC_SIGNED 40.96


# QA

In [ ]:
modelqa = distiller.models.create_model(False, "cifar10", "resnet56_cifar", True, None)
originalDictionary = copy.deepcopy(modelqa.state_dict())
opt = torch.optim.SGD(modelqa.parameters(), lr=0.01, momentum=0.5, weight_decay=0, nesterov = False) #can pass in any SGD. the arguments DO NOT matter
device = "cuda" if torch.cuda.is_available() else "cpu"
quantizer = distiller.quantization.QuantAwareTrainRangeLinearQuantizer(model = copy.deepcopy(modelqa), bits_activations=8, optimizer = copy.deepcopy(opt), bits_weights = 8,
                                                            mode=distiller.quantization.LinearQuantMode.ASYMMETRIC_UNSIGNED, per_channel_wts=True)
dummy_input = distiller.get_dummy_input(input_shape= modelqa.input_shape, device = device)
quantizer.prepare_model(dummy_input)

In [ ]:
modelDirectory = "/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Quantization Schedules/QAResultsANone/2020.12.08-233548"

mapping = {}
mapping2 = {}

for file in os.listdir(modelDirectory):
  # print(file)
  target = os.path.join(modelDirectory, "best.pth.tar")
  print(target)
  if os.path.exists(target):
    checkpoint = torch.load(target)
    quantizer.model.load_state_dict(checkpoint["state_dict"])
    sparsity = round(distiller.utils.model_sparsity((quantizer.model)), 2)
    acc = round(checkpoint["extras"]["best_top1"], 2)
    mapping[file] = str(acc)
    mapping2[file] = str(sparsity)
    str(sparsity)

df = pd.DataFrame.from_dict(mapping, orient = 'index')
df2 = pd.DataFrame.from_dict(mapping2, orient = 'index')

df

/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Quantization Schedules/QAResultsANone/2020.12.08-233548/best.pth.tar
/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Quantization Schedules/QAResultsANone/2020.12.08-233548/best.pth.tar
/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Quantization Schedules/QAResultsANone/2020.12.08-233548/best.pth.tar
/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Quantization Schedules/QAResultsANone/2020.12.08-233548/best.pth.tar
/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Quantization Schedules/QAResultsANone/2020.12.08-233548/best.pth.tar


,0
2020.12.08-233548.log,93.03
configs,93.03
events.out.tfevents.1607470558.f3dbda43d604,93.03
best.pth.tar,93.03
checkpoint.pth.tar,93.03


In [ ]:
checkpoint

{'arch': 'resnet56_cifar',
 'compression_sched': {'masks_dict': {'module.bn1.bias': None,
   'module.bn1.weight': None,
   'module.conv1.weight': None,
   'module.fc.bias': None,
   'module.fc.weight': None,
   'module.layer1.0.bn1.bias': None,
   'module.layer1.0.bn1.weight': None,
   'module.layer1.0.bn2.bias': None,
   'module.layer1.0.bn2.weight': None,
   'module.layer1.0.conv1.weight': None,
   'module.layer1.0.conv2.weight': None,
   'module.layer1.1.bn1.bias': None,
   'module.layer1.1.bn1.weight': None,
   'module.layer1.1.bn2.bias': None,
   'module.layer1.1.bn2.weight': None,
   'module.layer1.1.conv1.weight': None,
   'module.layer1.1.conv2.weight': None,
   'module.layer1.2.bn1.bias': None,
   'module.layer1.2.bn1.weight': None,
   'module.layer1.2.bn2.bias': None,
   'module.layer1.2.bn2.weight': None,
   'module.layer1.2.conv1.weight': None,
   'module.layer1.2.conv2.weight': None,
   'module.layer1.3.bn1.bias': None,
   'module.layer1.3.bn1.weight': None,
   'module.lay

In [ ]:
bits_activations = checkpoint['quantizer_metadata']["params"]["bits_activations"]
bits_weights = checkpoint['quantizer_metadata']["params"]["bits_weights"]
# clip_acts = checkpoint['quantizer_metadata']["params"]["clip_acts"]
mode = checkpoint['quantizer_metadata']["params"]["mode"]["activations"]
accur = round(checkpoint["extras"]["best_top1"], 2)
print(bits_activations, bits_weights, mode, accur)

14 14 ASYMMETRIC_UNSIGNED 93.03


In [ ]:
# pd.options.display.float_format = '{:.2f}'.format

# modelDirectory = "/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Quantization Schedules/QAResultsANone/2020.12.08-221150"

# #MUST ENSURE YOU HAVE THE SAME GPU BEFORE RUNNING
# # round to 2 decimal places
# df = pd.DataFrame()
# for file in os.listdir(modelDirectory):
#   target = os.path.join(modelDirectory, "best.pth.tar")
#   print(target)
#   if os.path.exists(target):
#       checkpoint = torch.load(target)
#       quantizer.model.load_state_dict(checkpoint["state_dict"])
#       stats = computeStatistics(modelqa)   
#       stats['Accur-Spar'] =  str(stats["Accuracy"]) + "-" + str(stats['Sparsity'])         
#       df_temp = pd.DataFrame.from_dict(stats, orient = 'index', columns = [file])
#       df = pd.concat([df, df_temp], axis=1)
#       print()
# df = df.T
# df

In [ ]:
folder = "/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/ComputeStatistics/Pruning/"
writer = pd.ExcelWriter('/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/ComputeStatistics/Pruning/Pruning.xlsx')

extension = 'csv'
all_filename_paths = [i for i in glob.glob(folder +'*.{}'.format(extension))]
for csv_path in all_filename_paths:
    csvfilename = csv_path.replace(folder,'')
    print(csvfilename)
    df = pd.read_csv(csv_path)
    if df.empty == True:
      print('DataFrame is empty')
    else:
      df.to_excel(writer,sheet_name=os.path.splitext(csvfilename)[0])
writer.save()

OneShotExpLR.csv
DataFrame is empty
OneShotExp40Epoch.csv
DataFrame is empty
OneShotElementBaseNoOptResults.csv
OneShotElementBaseOptResults.csv
TestSchedules.csv
DataFrame is empty
IterativeSensitivity.csv
IterativeSensitivitySchedules.csv
DataFrame is empty
SensitivityExperiments1.csv
SensitivitySchedules0.68Base.csv
DataFrame is empty
MSMG Schedules.csv
DataFrame is empty
SensitivityExperiments2.csv
DataFrame is empty
MSMG Results.csv
DataFrame is empty
FixedSensitivityExperiments.csv
MSMGResults.csv
RetrainResults1.csv
TuningExperiments.csv


In [ ]:
# microfade before beep
# chaing english ending to giberish (backgward text)
# lowpass earthquake,
# schurb tool before the english
# reverb on train 
# Command e, add a new monotrack, hold down options, duplicate and you can draw over section 
# select, audiosuite, gain, and make gain to very small 0 to english

In [ ]:
df.T

,Accuracy,Sparsity,Time,Model Parameters,Size_bitmap_LZMA,Size_bitmap_DEFLATED,Size_deltaEncoding_LZMA,Size_deltaEncoding_DEFLATED,SCR_bitmap_LZMA,SCR_bitmap_DEFLATED,SCR_deltaEncoding_LZMA,SCR_deltaEncoding_DEFLATED,ParameterCompressionRate,IndexSavingRate,Accur-Spar
2020.12.03-073428,10.00,100.00,2.82,855506,113913,105897,114413,106294,30.78,33.10,30.64,32.98,1.00,0.00,10.0-100.0
2020.12.03-080635,75.99,62.16,2.86,326494,1426748,1425424,1463509,1463460,2.46,2.46,2.40,2.40,2.62,1.62,75.99-62.16
2020.12.03-082420,10.00,100.00,2.80,855770,113595,105583,114086,105980,30.86,33.20,30.73,33.08,1.00,0.00,10.0-100.0
2020.12.03-082542,10.00,99.99,2.81,855494,114112,106095,114596,106487,30.72,33.04,30.59,32.92,1.00,0.00,10.0-99.99
2020.12.03-090147,10.00,99.99,2.79,855471,114140,106125,114625,106514,30.71,33.03,30.58,32.91,1.00,0.00,10.0-99.99
2020.12.03-090213,10.00,99.99,2.78,855711,113872,105873,114359,106265,30.79,33.11,30.66,32.99,1.00,0.00,10.0-99.99
2020.12.03-093914,10.00,100.00,2.77,855770,113652,105623,114086,105963,30.85,33.19,30.73,33.08,1.00,0.00,10.0-100.0


In [ ]:
df.T.to_csv("data.csv")
# df.to_csv("/content/drive/MyDrive/CS6787 - Final Project/OneShotElementBaseNoOptResults.csv")

In [ ]:
subfolders

['/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Pruning Schedules/.ipynb_checkpoints',
 '/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Pruning Schedules/OneShotExpLR',
 '/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Pruning Schedules/OneShotExp40Epoch',
 '/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Pruning Schedules/OneShotElementBaseNoOptResults',
 '/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Pruning Schedules/OneShotElementBaseOptResults',
 '/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Pruning Schedules/TestSchedules',
 '/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Pruning Schedules/IterativeSensitivity',
 '/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Pruning Schedules/IterativeSensitivitySchedules',
 '/content/drive/MyDrive/CS6787 -

In [ ]:
pd.options.display.float_format = '{:.4f}'.format

modelDirectory = "/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Pruning Schedules"
#MUST ENSURE YOU HAVE THE SAME GPU BEFORE RUNNING

subfolders = [f.name for f in os.scandir(modelDirectory) if f.is_dir()]

for experiments in subfolders:
  path_experiment = os.path.join(modelDirectory, experiments) 
  print(experiments)
  if os.path.exists(path_experiment):
    df = pd.DataFrame()
    for file in os.listdir(path_experiment):
      print(file)
      target = os.path.join(path_experiment, file, "best.pth.tar")
      if os.path.exists(target):
          print(":)")
          model = distiller.models.create_model(False, "cifar10", "resnet56_cifar", True, None)
          #load the file
          checkpoint = torch.load(target)
          model.load_state_dict(checkpoint["state_dict"])
          stats = computeStatistics(model)
          df_temp = pd.DataFrame.from_dict(stats, orient = 'index', columns = [file])
          df = pd.concat([df, df_temp], axis=1)
          # print()
          stats = computeStatistics(model)   
          stats['Accur-Spar'] =  str(stats["Accuracy"]) + "-" + str(stats['Sparsity'])         
          df_temp = pd.DataFrame.from_dict(stats, orient = 'index', columns = [file])
          df = pd.concat([df, df_temp], axis=1)
      print()
    df = df.T
    df.to_csv("/content/drive/MyDrive/CS6787 - Final Project/" + experiments + ".csv")

.ipynb_checkpoints
OneShotExpLR
threshold0-1ExpLR.yaml

threshold0ExpLR.yaml

threshold0-5ExpLR.yaml

threshold1ExpLR.yaml

OneShotExp40Epoch
threshold0-1ExpLR.yaml

threshold0-5ExpLR.yaml

threshold0ExpLR.yaml

threshold1ExpLR.yaml

OneShotElementBaseNoOptResults
90.85-52.2
:)

91.95-52.2
:)

89.35-85.94
:)

90.17-85.94
:)

90.02-82.6
:)

91.0-67.71
:)

91.3-67.71
:)

92.73-52.2
:)

92.83-52.2
:)

89.56-85.94
:)

91.24-85.94
:)

90.37-82.6
:)

91.51-82.6
:)

92.21-67.71
:)

92.68-67.71
:)

90.82-82.6
:)

93.0-52.2
:)

92.96-67.71
:)

92.15-82.6
:)

91.99-85.94
:)

92.99-52.2
:)

92.65-67.71
:)

91.5-82.6
:)

91.17-85.94
:)

93.12-52.2
:)

92.87-67.71
:)

92.49-82.6
:)

92.2-85.94
:)

93.07-52.2
:)

92.91-52.2
:)

93.23-52.2
:)

92.14-82.6
:)

91.6-85.94
:)

Results-20201202T035922Z-001.zip

mappingFinal.csv

OneShotElementBaseOptResults
90.98-52.46
:)

91.45-52.46
:)

88.57-86.37
:)

90.37-86.37
:)

89.85-83.01
:)

90.95-83.01
:)

90.96-68.05
:)

91.49-68.05
:)

92.55-52.46
:)

92.97-

In [ ]:
folder = "/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/ComputeStatistics/Pruning/"
writer = pd.ExcelWriter('/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/ComputeStatistics/Pruning/Pruning.xlsx')

extension = 'csv'
all_filename_paths = [i for i in glob.glob(folder +'*.{}'.format(extension))]
for csv_path in all_filename_paths:
    csvfilename = csv_path.replace(folder,'')
    print(csvfilename)
    df = pd.read_csv(csv_path)
    if df.empty == True:
      print('DataFrame is empty')
    else:
      df.to_excel(writer,sheet_name=os.path.splitext(csvfilename)[0])
writer.save()

OneShotExpLR.csv
DataFrame is empty
OneShotExp40Epoch.csv
DataFrame is empty
OneShotElementBaseNoOptResults.csv
TestSchedules.csv
DataFrame is empty
OneShotElementBaseOptResults.csv
IterativeSensitivitySchedules.csv
DataFrame is empty
IterativeSensitivity.csv
SensitivitySchedules0.68Base.csv
DataFrame is empty
SensitivityExperiments1.csv
SensitivityExperiments2.csv
DataFrame is empty
FixedSensitivityExperiments.csv
MSMG Results.csv
DataFrame is empty
MSMG Schedules.csv
DataFrame is empty
MSMGResults.csv
RetrainResults1.csv
TuningExperiments.csv


In [ ]:
# df = pd.read_csv(file)
organized = "/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Result CSV/Results Organize/Results-New.xlsx"
old = "/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Result CSV/Results Organize/Results-OG-Headless.xlsx"
spreadsheets = pd.ExcelFile(old).sheet_names

df_organized = pd.DataFrame(pd.read_excel(organized))
df_organized

,Schedule,Epochs,Base LR,Model,Accuracy,Sparcity,Time,Model Parameters,Size_bitmap_LZMA,Size_bitmap_DEFLATED,Size_deltaEncoding_LZMA,Size_deltaEncoding_DEFLATED,SCR_bitmap_LZMA,SCR_bitmap_DEFLATED,SCR_deltaEncoding_LZMA,SCR_deltaEncoding_DEFLATED,ParameterCompressionRate,IndexSavingRate,Accur-Spar
0,short epoch (42 epochs) half,85,0.01,Opt,92.91,39.51,2.836476,519327.0,2090286.0,2109149.0,2129632.0,2143822.0,1.677130,1.662131,1.646144,1.635249,1.647844,0.647844,92.91-39.51
1,short epoch (42 epochs) half,85,0.01,NaN,93.06,39.53,2.840591,519129.0,2087387.0,2108400.0,2127709.0,2143067.0,1.679460,1.662721,1.647632,1.635825,1.648473,0.648473,93.06-39.53
2,short epoch (42 epochs) half,85,0.03,NaN,92.83,42.17,2.813334,496672.0,2004794.0,2026878.0,2046557.0,2061973.0,1.748649,1.729597,1.712966,1.700159,1.723008,0.723008,92.83-42.17
3,short epoch (42 epochs) half,85,0.03,Opt,92.91,42.41,2.877814,496311.0,2006991.0,2025081.0,2044744.0,2060136.0,1.746735,1.731132,1.714485,1.701675,1.724262,0.724262,92.91-42.21
4,short epoch (82 epochs),125,0.03,NaN,93.12,44.48,2.835231,476987.0,1933706.0,1954841.0,1974286.0,1990220.0,1.812934,1.793334,1.775671,1.761455,1.794116,0.794116,93.12-44.48
5,short epoch (42 epochs) half,85,0.05,Opt,92.87,45.56,2.821330,467864.0,1902937.0,1921516.0,1942948.0,1957024.0,1.842248,1.824435,1.804311,1.791333,1.829100,0.829100,92.87-45.56
6,short epoch (42 epochs) half,85,0.05,NaN,92.67,45.66,2.813800,466946.0,1902514.0,1918031.0,1939969.0,1953734.0,1.842658,1.827750,1.807081,1.794350,1.832696,0.832696,92.67-45.66
7,short epoch (82 epochs),125,0.05,NaN,92.99,49.29,2.833893,436062.0,1787467.0,1804962.0,1827194.0,1840665.0,1.961257,1.942247,1.918615,1.904574,1.962496,0.962496,92.99-49.29
8,short epoch (42 epochs) half,85,0.08,Opt,93.14,49.57,2.898981,433684.0,1779054.0,1795886.0,1819816.0,1831626.0,1.970532,1.952063,1.926394,1.913973,1.973257,0.973257,93.14-49.57
9,short epoch (42 epochs) half,85,0.08,NaN,92.95,49.63,2.825884,433195.0,1778004.0,1794169.0,1818482.0,1830038.0,1.971695,1.953931,1.927807,1.915633,1.975484,0.975484,92.95-49.63


In [ ]:
df_old = pd.DataFrame(pd.read_excel(old))
df_old

,Schedule,Epochs,Base LR,Model,Accuracy,Sparcity
0,threshold1S,85,0.08,NaN,91.91,87.68
1,threshold1S,85,0.05,NaN,91.83,87.00
2,threshold0-5S,85,0.08,NaN,91.81,86.34
3,threshold1S,85,0.03,NaN,91.85,85.95
4,threshold0-5S,85,0.05,NaN,92.12,85.40
5,threshold0-5S,85,0.03,NaN,91.76,84.27
6,threshold1S,85,0.01,NaN,91.31,84.21
7,threshold0-5S,85,0.01,NaN,91.56,82.48
8,threshold0-1S,85,0.08,NaN,92.22,79.11
9,threshold0-1S,85,0.05,NaN,92.58,77.69


In [ ]:
df_old.merge(df_organized, how='left')

,Schedule,Epochs,Base LR,Model,Accuracy,Sparcity,Time,Model Parameters,Size_bitmap_LZMA,Size_bitmap_DEFLATED,Size_deltaEncoding_LZMA,Size_deltaEncoding_DEFLATED,SCR_bitmap_LZMA,SCR_bitmap_DEFLATED,SCR_deltaEncoding_LZMA,SCR_deltaEncoding_DEFLATED,ParameterCompressionRate,IndexSavingRate,Accur-Spar
0,threshold1S,85,0.08,NaN,91.91,87.68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,threshold1S,85,0.05,NaN,91.83,87.00,2.783136,115000.0,594612.0,595888.0,628946.0,635534.0,5.895747,5.883122,5.573900,5.516120,7.441478,6.441478,91.83-87.0
2,threshold0-5S,85,0.08,NaN,91.81,86.34,2.836116,120604.0,614752.0,617563.0,651219.0,657416.0,5.702596,5.676639,5.383261,5.332517,7.095702,6.095702,91.81-86.34
3,threshold1S,85,0.03,NaN,91.85,85.95,2.797660,123900.0,626233.0,629679.0,662330.0,669768.0,5.598047,5.567411,5.292954,5.234174,6.906941,5.906941,91.85-85.95
4,threshold0-5S,85,0.05,NaN,92.12,85.40,2.827439,128583.0,644683.0,647381.0,680048.0,687295.0,5.437838,5.415176,5.155051,5.100695,6.655390,5.655390,92.12-85.4
5,threshold0-5S,85,0.03,NaN,91.76,84.27,2.817664,138231.0,679544.0,683775.0,716319.0,724005.0,5.158874,5.126953,4.894023,4.842069,6.190869,5.190869,91.76-84.27
6,threshold1S,85,0.01,NaN,91.31,84.21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,threshold0-5S,85,0.01,NaN,91.56,82.48,2.811925,153481.0,735471.0,741607.0,773590.0,782055.0,4.766581,4.727143,4.531705,4.482654,5.575739,4.575739,91.56-82.48
8,threshold0-1S,85,0.08,NaN,92.22,79.11,2.825781,182155.0,841985.0,849823.0,882170.0,890515.0,4.163592,4.125191,3.973930,3.936691,4.698032,3.698032,92.22-79.11
9,threshold0-1S,85,0.05,NaN,92.58,77.69,2.834243,194278.0,885473.0,894520.0,926506.0,935056.0,3.959107,3.919065,3.783766,3.749168,4.404873,3.404873,92.58-77.69


In [ ]:
spreadsheets

['IterativePruning',
 'MSMG',
 'FurtherTraining',
 'FurtherTraining RetrainResults1',
 'QuantizationPost',
 'QuantizedTraining']

In [ ]:
df_organized = pd.DataFrame(pd.read_excel(organized, sheet_name = 'MSMG'))
df_organized

,Restore,Yaml,Epoch,Base LR,Accuracy,Sparcity,Time,Model Parameters,Size_bitmap_LZMA,Size_bitmap_DEFLATED,Size_deltaEncoding_LZMA,Size_deltaEncoding_DEFLATED,SCR_bitmap_LZMA,SCR_bitmap_DEFLATED,SCR_deltaEncoding_LZMA,SCR_deltaEncoding_DEFLATED,ParameterCompressionRate,IndexSavingRate,Accur-Spar,Unnamed: 0
0,0.3,threshold0S,100,0.03,92.96,63.99,2.777503,310880.0,1316440.0,1328811.0,1357140.0,1367630.0,2.663002,2.638210,2.583140,2.563326,2.752734,1.752734,92.96-63.99,2020.12.04-234942
1,0.5,threshold0S,100,0.03,92.94,64.68,2.796923,305034.0,1295533.0,1306863.0,1337163.0,1345715.0,2.705977,2.682517,2.621731,2.605070,2.805491,1.805491,92.94-64.68,2020.12.04-235738
2,0.7,threshold0S,100,0.03,92.92,65.62,2.840312,296989.0,1266276.0,1277526.0,1306688.0,1316462.0,2.768498,2.744118,2.682876,2.662957,2.881487,1.881487,92.92-65.62,2020.12.05-000526
3,0.3,threshold0S,100,0.08,93.07,67.61,2.790705,280097.0,1206438.0,1215231.0,1246212.0,1254322.0,2.905812,2.884787,2.813070,2.794882,3.055263,2.055263,93.07-67.61,2020.12.04-230013
4,0.5,threshold0S,100,0.08,92.68,69.29,2.852023,265768.0,1155067.0,1162380.0,1194120.0,1201562.0,3.035046,3.015952,2.935787,2.917604,3.219989,2.219989,92.68-69.29,2020.12.04-230720
5,0.7,threshold0S,100,0.08,92.74,71.34,2.791946,248271.0,1095539.0,1101351.0,1134316.0,1140907.0,3.199961,3.183074,3.090569,3.072715,3.446919,2.446919,92.74-71.34,2020.12.04-231540
6,0.3,threshold0-1S,100,0.03,92.52,75.53,2.808366,212619.0,953696.0,962777.0,993688.0,1003264.0,3.675890,3.641219,3.527950,3.494277,4.024899,3.024899,92.52-75.53,2020.12.05-012744
7,0.5,threshold0-1S,100,0.03,92.69,76.20,2.815063,206902.0,932238.0,941467.0,972691.0,981792.0,3.760501,3.723638,3.604107,3.570697,4.136113,3.136113,92.69-76.2,2020.12.05-013904
8,0.7,threshold0-1S,100,0.03,92.43,77.08,2.804693,199426.0,905726.0,913530.0,945408.0,954049.0,3.870577,3.837512,3.708115,3.674530,4.291166,3.291166,92.43-77.08,2020.12.05-014526
9,0.3,threshold0-1S,100,0.08,92.54,78.49,2.804368,187448.0,861266.0,869475.0,901962.0,910091.0,4.070382,4.031953,3.886729,3.852013,4.565373,3.565373,92.54-78.49,2020.12.05-003905


In [ ]:
for sheet in spreadsheets:
  print(sheet)
  df_organized = pd.DataFrame(pd.read_excel(organized, sheet_name = sheet))
  df_old = pd.DataFrame(pd.read_excel(old, sheet_name = sheet))
  df = df_old.merge(df_organized, how='left')
  df.to_csv("/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Result CSV/Results Organize/" + sheet + ".csv")

IterativePruning
MSMG
FurtherTraining
FurtherTraining RetrainResults1
QuantizationPost
QuantizedTraining


In [6]:
old = "/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Result CSV/Results Organize/Results.xlsx"
spreadsheets = pd.ExcelFile(old).sheet_names
spreadsheets

['OneShot',
 'OneShotBase',
 'StorageComparison',
 'IterativePruning',
 'MSMG',
 'FurtherTraining',
 'FurtherTraining RetrainResults1',
 'QuantizationPost',
 'QuantizedTraining',
 'PruningQAFurther',
 'Pruning+Quant',
 'OneShotElementBaseNoOptResults',
 'OneShotElementBaseOptResults',
 'JeremyPost']

In [16]:
df_organized = pd.read_csv("/content/drive/MyDrive/CS6787 - Final Project/QASparseResults.csv")
df_organized

,Unnamed: 0,Accuracy,Mode,Bits activations,Bits weights,Model Parameters,Size_bitmap_LZMA,Size_bitmap_DEFLATED,Size_deltaEncoding_LZMA,Size_deltaEncoding_DEFLATED,Size_bitarray_LZMA,Size_bitarray_DEFLATED,Size_zip_LZMA,Size_zip_DEFLATED,SCR_bitmap_LZMA,SCR_bitmap_DEFLATED,SCR_deltaEncoding_LZMA,SCR_deltaEncoding_DEFLATED,scr_bitarray_LZMA,scr_bitarray_DEFLATED,scr_zip_LZMA,scr_zip_DEFLATED,ParameterCompressionRate,IndexSavingRate
0,2020.12.10-222727,92.57,SYMMETRIC,8,8,107866,341451,341451,410552,398591,973080,961168,994051,1208348,10.266018,10.589613,8.538119,8.794333,3.602316,3.646961,3.526320,2.900937,7.933640,6.933640
1,2020.12.10-225607,92.58,SYMMETRIC,12,12,107866,391833,391833,497113,484383,1403750,1385856,1025026,1258591,8.946010,9.208010,7.051399,7.236716,2.497127,2.529370,3.419759,2.785132,7.933640,6.933640
2,2020.12.10-232425,92.57,SYMMETRIC,12,12,107866,391878,391878,497225,484405,1403832,1385873,1018127,1260340,8.944983,9.207913,7.049810,7.236387,2.496981,2.529339,3.442932,2.781267,7.933640,6.933640
3,2020.12.11-000517,83.09,SYMMETRIC,4,4,152558,320348,320348,353233,341667,541249,536022,1047473,1287290,10.942294,11.243864,9.923597,10.259528,6.476394,6.539549,3.346475,2.723040,5.609473,4.609473
4,2020.12.11-000527,84.02,SYMMETRIC,4,4,99834,284077,284077,318211,307270,540096,534790,801309,1006587,12.339408,12.838482,11.015779,11.408019,6.490220,6.554614,4.374520,3.482403,8.571929,7.571929
5,2020.12.11-003240,91.67,SYMMETRIC,6,6,100165,310143,310143,360460,349205,757028,748027,879839,1090233,11.302341,11.697145,9.724635,10.038064,4.630399,4.686117,3.984072,3.215223,8.543603,7.543603
6,2020.12.11-003342,92.04,SYMMETRIC,6,6,153234,361518,361518,406605,394509,758605,749773,1158987,1413076,9.696176,10.014948,8.621001,8.885328,4.620774,4.675204,3.024488,2.480646,5.584727,4.584727
7,2020.12.11-025542,83.13,SYMMETRIC,4,4,107829,287211,287211,323286,312153,540719,535478,832632,1041006,12.204762,12.708388,10.842851,11.229564,6.482742,6.546192,4.209953,3.367264,7.936362,6.936362
8,2020.12.11-030731,83.15,SYMMETRIC,4,4,152556,319750,319750,353329,341662,541282,535990,1048148,1287918,10.962758,11.305075,9.920901,10.259678,6.476000,6.539939,3.344320,2.721712,5.609547,4.609547
9,2020.12.11-030739,83.77,SYMMETRIC,4,4,211345,362097,362097,391447,379399,541414,536350,1294133,1572230,9.680671,9.955247,8.954832,9.239197,6.474421,6.535550,2.708641,2.229535,4.049161,3.049161


In [13]:
df_old.columns = df_old.iloc[0]
df_old = df_old[1:]
df_old

,Base Model,Mode,Clip Act,Bits,Epochs,LR,PreQA Sparcity,PreQA Accuracy,Sparcity,Accuracy,NaN,NaN
1,TuningExperiments/2020.12.05-221020,S,AVG,4,25,0.0001,89.25,92.25,89.06,83.8,NaN,NaN
2,TuningExperiments/2020.12.05-184050,S,AVG,4,25,0.0001,88.36,92.63,88.15,83.13,NaN,NaN
3,OneShotElementBaseNoOptResults/92.49-82.6,S,AVG,4,25,0.0001,82.6,92.49,82.86,83.15,NaN,NaN
4,MSMGResults/2020.12.05-013904,S,AVG,4,25,0.0001,76.2,92.69,76.02,83.77,NaN,NaN
5,TuningExperiments/2020.12.05-221020,S,AVG,6,25,0.0001,89.25,92.25,89.04,91.85,NaN,NaN
6,TuningExperiments/2020.12.05-184050,S,AVG,6,25,0.0001,88.36,92.63,88.15,91.9,NaN,NaN
7,OneShotElementBaseNoOptResults/92.49-82.6,S,AVG,6,25,0.0001,82.6,92.49,82.82,92.15,NaN,NaN
8,MSMGResults/2020.12.05-013904,S,AVG,6,25,0.0001,76.2,92.69,76.02,91.82,NaN,NaN
9,TuningExperiments/2020.12.05-221020,S,AVG,8,25,0.0001,89.25,92.25,89.04,92.13,NaN,Using LZMA and Bitmap Encoding
10,TuningExperiments/2020.12.05-184050,S,AVG,8,25,0.0001,88.36,92.63,88.15,92.56,7.93x,10.59x


In [17]:
df_old = pd.DataFrame(pd.read_excel(old, sheet_name = "PruningQAFurther"))
df = df_old.merge(df_organized, how='left')
# df.to_csv("/content/drive/MyDrive/CS6787 - Final Project/10BitResultsOrganized.csv")
df

MergeError: ignored

In [ ]:
folder = "/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Result CSV/Results Organize/"
writer = pd.ExcelWriter("/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Result CSV/Results Organize/Organize.xls")

extension = 'csv'
all_filename_paths = [i for i in glob.glob(folder +'*.{}'.format(extension))]
for csv_path in all_filename_paths:
    csvfilename = csv_path.replace(folder,'')
    print(csvfilename)
    df = pd.read_csv(csv_path)
    if df.empty == True:
      print('DataFrame is empty')
    else:
      df.to_excel(writer,sheet_name=os.path.splitext(csvfilename)[0])
writer.save()

IterativePruning.csv
MSMG.csv
FurtherTraining.csv
FurtherTraining RetrainResults1.csv
QuantizationPost.csv
QuantizedTraining.csv


# Testing

In [ ]:
#testing storage functions
checkpoint = torch.load("/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Pruning Schedules/TuningExperiments/2020.12.05-184050/best.pth.tar", map_location = "cpu")
# checkpoint = torch.load("/content/drive/MyDrive/Colab Notebooks/6787 Notebooks/Google Colab Scripts/Schedules/Pruning Schedules/FixedSensitivityExperiments/2020.12.05-044938/best.pth.tar", map_location = "cpu")
# checkpoint = torch.load("/content/drive/MyDrive/Colab Notebooks/6787 Notebooks/Google Colab Scripts/Schedules/Pruning Schedules/FixedSensitivityExperiments/2020.12.04-142239/best.pth.tar", map_location = "cpu")

print(compressDictionary(checkpoint["state_dict"], "bitmap"))
print(compressDictionary(checkpoint["state_dict"], "deltaEncoding"))

# print(computeStatePyTorch(checkpoint["state_dict"]))

# tensor = checkpoint["state_dict"]["module.conv1.weight"].cpu()
# print(computeStorageTensor("test", tensor))
# print(computeStorageTensorDelta("test", tensor))
# print(np.diff([5,1,2,3]))
# print(computeStorageTensor("test", tensor))
# print(deltaEncodeIndicesValues([2,10,35,42,60], [1,2,3,4,5]))
# print(convertDeltaIndicesToBits(np.array([1,2,3,4,5,6,7])))
# print(np.bitwise_and(2,2))
# print(2&2)
# torch.save(tensor, "original")
# print(os.path.getsize("original"))

[557617, 553397]
[588565, 591048]


In [ ]:
model = distiller.models.create_model(False, "cifar10", "resnet56_cifar", True, None)
checkpoint = torch.load("/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Pruning Schedules/TuningExperiments/2020.12.05-184050/best.pth.tar", map_location = "cpu")
model.load_state_dict(checkpoint["state_dict"])
stats = computeStatistics(model)

In [ ]:
stats

{'Accuracy': 92.63,
 'IndexSavingRate': 7.274465060963228,
 'Model Parameters': 103423,
 'ParameterCompressionRate': 8.274465060963228,
 'SCR_bitmap_DEFLATED': 6.334840991187159,
 'SCR_bitmap_LZMA': 6.286899430971437,
 'SCR_deltaEncoding_DEFLATED': 5.931298304029453,
 'SCR_deltaEncoding_LZMA': 5.956320882145557,
 'Size_bitmap_DEFLATED': 553397,
 'Size_bitmap_LZMA': 557617,
 'Size_deltaEncoding_DEFLATED': 591048,
 'Size_deltaEncoding_LZMA': 588565,
 'Sparsity': 88.35507525507808,
 'Time': 2.8572661876678467}

In [ ]:
df_temp = pd.DataFrame.from_dict(stats, orient = 'index', columns = ["Model"])
df_temp

,Model
Accuracy,92.630000
Sparsity,88.355075
Time,2.857266
Model Parameters,103423.000000
Size_bitmap_LZMA,557617.000000
Size_bitmap_DEFLATED,553397.000000
Size_deltaEncoding_LZMA,588565.000000
Size_deltaEncoding_DEFLATED,591048.000000
SCR_bitmap_LZMA,6.286899
SCR_bitmap_DEFLATED,6.334841


In [ ]:
df = pd.DataFrame()
# df = df.append(stats, ignore_index=True, row = ["ModelA"])
df = pd.concat([df, df_temp], axis=1)
df

,Model
Accuracy,92.630000
Sparsity,88.355075
Time,2.857266
Model Parameters,103423.000000
Size_bitmap_LZMA,557617.000000
Size_bitmap_DEFLATED,553397.000000
Size_deltaEncoding_LZMA,588565.000000
Size_deltaEncoding_DEFLATED,591048.000000
SCR_bitmap_LZMA,6.286899
SCR_bitmap_DEFLATED,6.334841
